# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [34]:
problem = '''A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a proportional critical element, 0.4
and 0.2, respectively. That is to say, each litre of C1 contains 0.4 l of the critical
element. To obtain GCA, the mixture must proportionally contain at least a 0.3
fraction of the critical element. Another requirement is that the quantity of the
critical element is seen in GCB, an 0.3 fraction at the most.
Furthermore, the minimum ratio of C1 with C2 in product GCC must be 0.3.

The profit expected for the sale of each litre of GCA, GCB and GCC is $120, $135
and $155, respectively.

Optimise the production planning of this firm. '''

## 2. Generate the mathematical model

In [35]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [36]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Chemical Product Production

**Parameters:**

* **Availability of components:**
    * `A_C1` = 10,000 l (available liters of C1)
    * `A_C2` = 15,000 l (available liters of C2)
* **Minimum demand:**
    * `D_GCA` = 6,000 l (minimum demand of GCA)
    * `D_GCB` = 7,000 l (minimum demand of GCB)
    * `D_GCC` = 9,000 l (minimum demand of GCC)
* **Critical element proportion in components:**
    * `CE_C1` = 0.4 (proportion of critical element in C1)
    * `CE_C2` = 0.2 (proportion of critical element in C2)
* **Critical element requirement in products:**
    * `CE_GCA_min` = 0.3 (minimum proportion of critical element in GCA)
    * `CE_GCB_max` = 0.3 (maximum proportion of critical element in GCB)
* **Component ratio in GCC:**
    * `R_GCC` = 0.3 (minimum ratio of C1 to C2 in GCC)
* **Profit per liter:**
    * `P_GCA` = $120 (profit per liter of GCA)
    * `P_GCB` = $135 (profit per liter of GCB)
    * `P_GCC` = $155 (profit per liter of GCC)

**Decision Variables:**

* `GCA_C1`: Liters of C1 used in GCA
* `GCA_C2`: Liters of C2 used in GCA
* `GCB_C1`: Liters of C1 used in GCB
* `GCB_C2`: Liters of C2 used in GCB
* `GCC_C1`: Liters of C1 used in GCC
* `GCC_C2`: Liters of C2 used in GCC

**Objective Function:**

Maximize total profit:

```
Maximize Z = P_GCA*(GCA_C1 + GCA_C2) + P_GCB*(GCB_C1 + GCB_C2) + P_GCC*(GCC_C1 + GCC_C2)
```

**Constraints:**

1. **Component availability:**
    * `GCA_C1 + GCB_C1 + GCC_C1 <= A_C1` 
    * `GCA_C2 + GCB_C2 + GCC_C2 <= A_C2`
2. **Minimum demand:**
    * `GCA_C1 + GCA_C2 >= D_GCA`
    * `GCB_C1 + GCB_C2 >= D_GCB`
    * `GCC_C1 + GCC_C2 >= D_GCC`
3. **Critical element proportion in GCA:**
    * `(CE_C1 * GCA_C1 + CE_C2 * GCA_C2) / (GCA_C1 + GCA_C2) >= CE_GCA_min`
4. **Critical element proportion in GCB:**
    * `(CE_C1 * GCB_C1 + CE_C2 * GCB_C2) / (GCB_C1 + GCB_C2) <= CE_GCB_max`
5. **Component ratio in GCC:**
    * `GCC_C1 / GCC_C2 >= R_GCC`
6. **Non-negativity:**
    * `GCA_C1, GCA_C2, GCB_C1, GCB_C2, GCC_C1, GCC_C2 >= 0`

This model can be solved using linear programming solvers to find the optimal production plan that maximizes profit while meeting all the constraints. 


## 3. Generate the pyomo code

In [37]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [38]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define sets (optional, can be useful for larger problems)
components = ["C1", "C2"]
products = ["GCA", "GCB", "GCC"]

# Parameters (using the provided data)
A = {"C1": 10000, "C2": 15000}
D = {"GCA": 6000, "GCB": 7000, "GCC": 9000}
CE_C = {"C1": 0.4, "C2": 0.2}
CE_prod = {"GCA": {"min": 0.3}, "GCB": {"max": 0.3}}  # Note: Using dictionary for min/max
R_GCC = 0.3
P = {"GCA": 120, "GCB": 135, "GCC": 155}

# Decision Variables
model.X = pyo.Var(products, components, domain=pyo.NonNegativeReals)

# Objective Function
def objective_rule(model):
    return sum(P[p] * sum(model.X[p, c] for c in components) for p in products)
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
# 1. Component Availability
model.component_availability = pyo.ConstraintList()
for c in components:
    model.component_availability.add(
        sum(model.X[p, c] for p in products) <= A[c]
    )

# 2. Minimum Demand
model.demand = pyo.ConstraintList()
for p in products:
    model.demand.add(sum(model.X[p, c] for c in components) >= D[p])

# 3 & 4. Critical Element Proportion
model.critical_element = pyo.ConstraintList()
for p in products:
    total_prod = sum(model.X[p, c] for c in components)
    if 'min' in CE_prod[p]:
        model.critical_element.add(
            sum(CE_C[c] * model.X[p, c] for c in components) >= CE_prod[p]['min'] * total_prod
        )
    if 'max' in CE_prod[p]:
        model.critical_element.add(
            sum(CE_C[c] * model.X[p, c] for c in components) <= CE_prod[p]['max'] * total_prod
        )

# 5. Component Ratio in GCC
model.component_ratio = pyo.Constraint(
    expr=model.X["GCC", "C1"] >= R_GCC * model.X["GCC", "C2"]
)

# Solve the model
solver = pyo.SolverFactory('glpk') # You can use other solvers like 'gurobi', 'cplex' etc.
solver.solve(model)

# Print the results
print("Profit:", pyo.value(model.profit))
for p in products:
    for c in components:
        print(f"{p}_{c}: {pyo.value(model.X[p, c])}")
```

**Explanation and Sample Data:**

1.  **Parameters:** I used the data you provided as parameters for the model.
2.  **Sets:** While optional, I included sets for components and products to make the code more organized.
3.  **Decision Variables:**  `model.X` is a 2D decision variable indexed by product and component, representing the liters of each component used in each product.
4.  **Constraints:**  Each constraint is implemented using Pyomo's `ConstraintList` or `Constraint` objects, ensuring the solution meets all the requirements.
5.  **Solver:** The code uses the open-source solver "glpk" - you can replace it with other solvers you have available. 

This code will provide the optimal production plan to maximize profit while adhering to the constraints. Remember that the actual optimal solution will depend on the data you provide for the parameters. 


## 4. Input problem data and try running the generated code

In [39]:
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define sets (optional, can be useful for larger problems)
components = ["C1", "C2"]
products = ["GCA", "GCB", "GCC"]

# Parameters (using the provided data)
A = {"C1": 10000, "C2": 15000}
D = {"GCA": 6000, "GCB": 7000, "GCC": 9000}
CE_C = {"C1": 0.4, "C2": 0.2}
CE_prod = {"GCA": {"min": 0.3}, "GCB": {"max": 0.3}}  # Note: Using dictionary for min/max
R_GCC = 0.3
P = {"GCA": 120, "GCB": 135, "GCC": 155}

# Decision Variables
model.X = pyo.Var(products, components, domain=pyo.NonNegativeReals)

# Objective Function
def objective_rule(model):
    return sum(P[p] * sum(model.X[p, c] for c in components) for p in products)
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
# 1. Component Availability
model.component_availability = pyo.ConstraintList()
for c in components:
    model.component_availability.add(
        sum(model.X[p, c] for p in products) <= A[c]
    )

# 2. Minimum Demand
model.demand = pyo.ConstraintList()
for p in products:
    model.demand.add(sum(model.X[p, c] for c in components) >= D[p])

# 3 & 4. Critical Element Proportion
model.critical_element = pyo.ConstraintList()
for p in products:
    total_prod = sum(model.X[p, c] for c in components)
    if 'min' in CE_prod[p]:
        model.critical_element.add(
            sum(CE_C[c] * model.X[p, c] for c in components) >= CE_prod[p]['min'] * total_prod
        )
    if 'max' in CE_prod[p]:
        model.critical_element.add(
            sum(CE_C[c] * model.X[p, c] for c in components) <= CE_prod[p]['max'] * total_prod
        )

# 5. Component Ratio in GCC
model.component_ratio = pyo.Constraint(
    expr=model.X["GCC", "C1"] >= R_GCC * model.X["GCC", "C2"]
)

# Solve the model
solver = pyo.SolverFactory('glpk') # You can use other solvers like 'gurobi', 'cplex' etc.
solver.solve(model)

# Print the results
print("Profit:", pyo.value(model.profit))
for p in products:
    for c in components:
        print(f"{p}_{c}: {pyo.value(model.X[p, c])}")

KeyError: 'GCC'

## 5. Correct the code to verify model viability (optional)

In [40]:
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define sets (optional, can be useful for larger problems)
components = ["C1", "C2"]
products = ["GCA", "GCB", "GCC"]
products2 = ["GCA", "GCB"]

# Parameters (using the provided data)
A = {"C1": 10000, "C2": 15000}
D = {"GCA": 6000, "GCB": 7000, "GCC": 9000}
CE_C = {"C1": 0.4, "C2": 0.2}
CE_prod = {"GCA": {"min": 0.3}, "GCB": {"max": 0.3}}  # Note: Using dictionary for min/max
R_GCC = 0.3
P = {"GCA": 120, "GCB": 135, "GCC": 155}

# Decision Variables
model.X = pyo.Var(products, components, domain=pyo.NonNegativeReals)

# Objective Function
def objective_rule(model):
    return sum(P[p] * sum(model.X[p, c] for c in components) for p in products)
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
# 1. Component Availability
model.component_availability = pyo.ConstraintList()
for c in components:
    model.component_availability.add(
        sum(model.X[p, c] for p in products) <= A[c]
    )

# 2. Minimum Demand
model.demand = pyo.ConstraintList()
for p in products:
    model.demand.add(sum(model.X[p, c] for c in components) >= D[p])

# 3 & 4. Critical Element Proportion
model.critical_element = pyo.ConstraintList()
for p in products2:
    total_prod = sum(model.X[p, c] for c in components)
    if 'min' in CE_prod[p]:
        model.critical_element.add(
            sum(CE_C[c] * model.X[p, c] for c in components) >= CE_prod[p]['min'] * total_prod
        )
    if 'max' in CE_prod[p]:
        model.critical_element.add(
            sum(CE_C[c] * model.X[p, c] for c in components) <= CE_prod[p]['max'] * total_prod
        )

# 5. Component Ratio in GCC
model.component_ratio = pyo.Constraint(
    expr=model.X["GCC", "C1"] >= R_GCC * model.X["GCC", "C2"]
)

# Solve the model
solver = pyo.SolverFactory('glpk') # You can use other solvers like 'gurobi', 'cplex' etc.
solver.solve(model)

# Print the results
print("Profit:", pyo.value(model.profit))
for p in products:
    for c in components:
        print(f"{p}_{c}: {pyo.value(model.X[p, c])}")

Profit: 3525000.0
GCA_C1: 3730.76923076923
GCA_C2: 2269.23076923077
GCB_C1: 3500.0
GCB_C2: 3500.0
GCC_C1: 2769.23076923077
GCC_C2: 9230.76923076923


## 6. Printing the outputs as strings, so they can be saved.

In [41]:
print(response.text)

## Mathematical Optimization Model for Chemical Product Production

**Parameters:**

* **Availability of components:**
    * `A_C1` = 10,000 l (available liters of C1)
    * `A_C2` = 15,000 l (available liters of C2)
* **Minimum demand:**
    * `D_GCA` = 6,000 l (minimum demand of GCA)
    * `D_GCB` = 7,000 l (minimum demand of GCB)
    * `D_GCC` = 9,000 l (minimum demand of GCC)
* **Critical element proportion in components:**
    * `CE_C1` = 0.4 (proportion of critical element in C1)
    * `CE_C2` = 0.2 (proportion of critical element in C2)
* **Critical element requirement in products:**
    * `CE_GCA_min` = 0.3 (minimum proportion of critical element in GCA)
    * `CE_GCB_max` = 0.3 (maximum proportion of critical element in GCB)
* **Component ratio in GCC:**
    * `R_GCC` = 0.3 (minimum ratio of C1 to C2 in GCC)
* **Profit per liter:**
    * `P_GCA` = $120 (profit per liter of GCA)
    * `P_GCB` = $135 (profit per liter of GCB)
    * `P_GCC` = $155 (profit per liter of GCC)

**Dec

In [42]:
print(response2.text)

```python
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define sets (optional, can be useful for larger problems)
components = ["C1", "C2"]
products = ["GCA", "GCB", "GCC"]

# Parameters (using the provided data)
A = {"C1": 10000, "C2": 15000}
D = {"GCA": 6000, "GCB": 7000, "GCC": 9000}
CE_C = {"C1": 0.4, "C2": 0.2}
CE_prod = {"GCA": {"min": 0.3}, "GCB": {"max": 0.3}}  # Note: Using dictionary for min/max
R_GCC = 0.3
P = {"GCA": 120, "GCB": 135, "GCC": 155}

# Decision Variables
model.X = pyo.Var(products, components, domain=pyo.NonNegativeReals)

# Objective Function
def objective_rule(model):
    return sum(P[p] * sum(model.X[p, c] for c in components) for p in products)
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
# 1. Component Availability
model.component_availability = pyo.ConstraintList()
for c in components:
    model.component_availability.add(
        sum(model.X[p, c] for p in products) <= A[c]
    )

# 2